In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
pd. set_option("display.max_columns", None) 

In [3]:
def clean_age_rating(age_rating):
    if (age_rating.startswith('PEGI 3')):
        return 'PEGI 3'
    elif (age_rating.startswith('PEGI 16')):
        return 'PEGI 16'
    elif (age_rating.startswith('PEGI 12')):
        return 'PEGI 12'
    elif (age_rating.startswith('PEGI 18')):
        return 'PEGI 18'

In [4]:
def clean_price(price):
    price = str(price)
    if (price.startswith('£')):
        price = price.replace('£', '')
    return float(price)

In [5]:
def clean_download_count(no_of_downloads):
    return no_of_downloads.replace(' ', '')

In [6]:
def clean_app_rating_count(app_rating_count):
    app_rating_count = str(app_rating_count)
    no_of_char = len(app_rating_count)
    if app_rating_count[no_of_char-1] == 'K':
        return int(float(app_rating_count[:no_of_char-1])*1000)
    if app_rating_count[no_of_char-1] == 'M':
        return int(float(app_rating_count[:no_of_char-1])*1000000)
    return float(app_rating_count)

In [7]:
def clean_android_data(path, category):
    files = glob.glob(os.path.join(path , "*.csv"))
    dfs = []
    for filename in files:
        df = pd.read_csv(filename)
        dfs.append(df)

    df = pd.concat(dfs, axis=0, ignore_index=True)
    df['Age Rating'] = df['Age Rating'].apply(clean_age_rating)
    df['Downloads'] = df['Downloads'].apply(clean_download_count)
    df['Rating Count'] = df['Rating Count'].apply(clean_app_rating_count)
    rating_na_indexes = df[df['Rating Count'].isna()].index
    df.loc[rating_na_indexes, 'Rating Count'] = 0
    df['Rating Count'] = df['Rating Count'].astype('int')
    df['Price'] = df['Price'].apply(clean_price)
    price_na_indexes = df[df['Price'].isna()].index
    df.loc[price_na_indexes, 'Price'] = 0
    df['Age Rating'] = df['Age Rating'].replace(np.nan, 'None')
    df['Average Rating'] = df['Average Rating'].replace(np.nan, 0)
    df['Application Type'] = category
    return df

In [8]:
fitness_df = clean_android_data('./../android/fitness/', 'FITNESS')

In [9]:
medical_df = clean_android_data('./../android/medical/', 'MEDICAL')

In [10]:
concatenated_df = pd.concat([medical_df, fitness_df], axis=0, ignore_index=True)

In [11]:
concatenated_df[:5]

,App Name,Age Rating,Price,InApp Purchase,Average Rating,Rating Count,App Link,Last Updated,Released Date,Downloads,Version,Country Code,Application Type
0,TraceTogether,PEGI 3,0.00,False,2.9,57100,https://play.google.com/store/apps/details?id=...,"May 31, 2022","Mar 9, 2020","1,000,000+",2.14.2,FO,MEDICAL
1,Blood Pressure Analyze,PEGI 3,0.00,True,3.9,12300,https://play.google.com/store/apps/details?id=...,"Sep 9, 2021","Apr 1, 2019","1,000,000+",1.3.2,FO,MEDICAL
2,Monash University FODMAP diet,PEGI 3,5.60,False,3.5,2470,https://play.google.com/store/apps/details?id=...,"Jan 18, 2022","Nov 6, 2013","100,000+",3.0.9,FO,MEDICAL
3,Squeezy: NHS Pelvic Floor App,PEGI 3,2.99,False,4.6,3250,https://play.google.com/store/apps/details?id=...,"Mar 4, 2019","Sep 17, 2014","50,000+",Varies with device,FO,MEDICAL
4,Muscle Trigger Point Anatomy,PEGI 3,2.69,False,4.5,1590,https://play.google.com/store/apps/details?id=...,"Jul 30, 2019","Jan 27, 2011","50,000+",2.4.8,FO,MEDICAL


In [12]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12242 entries, 0 to 12241
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   App Name          12242 non-null  object 
 1   Age Rating        12242 non-null  object 
 2   Price             12242 non-null  float64
 3   InApp Purchase    12242 non-null  bool   
 4   Average Rating    12242 non-null  float64
 5   Rating Count      12242 non-null  int64  
 6   App Link          12242 non-null  object 
 7   Last Updated      12242 non-null  object 
 8   Released Date     11845 non-null  object 
 9   Downloads         12242 non-null  object 
 10  Version           12241 non-null  object 
 11  Country Code      12242 non-null  object 
 12  Application Type  12242 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(9)
memory usage: 1.1+ MB


#### Write to a single fine

In [13]:
concatenated_df.to_csv('./android-fitness-and-medical-cleaned-data.csv')

#### Some apps are also available in some countries. So remove duplicate and add two new rows. One for count of countries and onr for the country codes

In [14]:
def get_unique_app_with_link():
    copied_df = concatenated_df.copy()
    copied_df.drop_duplicates(subset='App Link', keep='first', inplace=True)
    copied_df.drop(columns='Country Code', axis=1, inplace=True)
    for index in copied_df.index:
        link = copied_df.at[index, 'App Link']
        unique_countries = (concatenated_df[concatenated_df['App Link'] == link]['Country Code'].unique())
        copied_df.at[index, 'Country Count'] = len(unique_countries)
        copied_df.at[index, 'Countries'] = ','.join(unique_countries)
    copied_df['Country Count'] = copied_df['Country Count'].astype('int')
    copied_df.to_csv('./android-fitness-and-medical-cleaned-data(no-duplicate).csv')
    return copied_df

In [15]:
unique_df = get_unique_app_with_link()

In [16]:
unique_df[:5]

,App Name,Age Rating,Price,InApp Purchase,Average Rating,Rating Count,App Link,Last Updated,Released Date,Downloads,Version,Application Type,Country Count,Countries
0,TraceTogether,PEGI 3,0.00,False,2.9,57100,https://play.google.com/store/apps/details?id=...,"May 31, 2022","Mar 9, 2020","1,000,000+",2.14.2,MEDICAL,31,"FO,HR,SM,GI,EE,SI,GL,SK,PF,SJ,IS,VA,BG,WF,LI,A..."
1,Blood Pressure Analyze,PEGI 3,0.00,True,3.9,12300,https://play.google.com/store/apps/details?id=...,"Sep 9, 2021","Apr 1, 2019","1,000,000+",1.3.2,MEDICAL,32,"FO,HR,SM,GI,EE,SI,GL,SK,PF,SJ,IS,VA,BG,WF,LI,A..."
2,Monash University FODMAP diet,PEGI 3,5.60,False,3.5,2470,https://play.google.com/store/apps/details?id=...,"Jan 18, 2022","Nov 6, 2013","100,000+",3.0.9,MEDICAL,46,"FO,PT,HR,IT,DE,FI,EE,HU,SI,ES,GL,IE,RO,SK,PF,S..."
3,Squeezy: NHS Pelvic Floor App,PEGI 3,2.99,False,4.6,3250,https://play.google.com/store/apps/details?id=...,"Mar 4, 2019","Sep 17, 2014","50,000+",Varies with device,MEDICAL,45,"FO,PT,HR,SM,GI,FI,EE,HU,SI,GL,IE,RO,SK,PF,SJ,I..."
4,Muscle Trigger Point Anatomy,PEGI 3,2.69,False,4.5,1590,https://play.google.com/store/apps/details?id=...,"Jul 30, 2019","Jan 27, 2011","50,000+",2.4.8,MEDICAL,42,"FO,PT,HR,IT,DE,FI,EE,HU,SI,GL,IE,RO,SK,PF,SJ,I..."


In [17]:
unique_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1571 entries, 0 to 12239
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   App Name          1571 non-null   object 
 1   Age Rating        1571 non-null   object 
 2   Price             1571 non-null   float64
 3   InApp Purchase    1571 non-null   bool   
 4   Average Rating    1571 non-null   float64
 5   Rating Count      1571 non-null   int64  
 6   App Link          1571 non-null   object 
 7   Last Updated      1571 non-null   object 
 8   Released Date     1317 non-null   object 
 9   Downloads         1571 non-null   object 
 10  Version           1570 non-null   object 
 11  Application Type  1571 non-null   object 
 12  Country Count     1571 non-null   int64  
 13  Countries         1571 non-null   object 
dtypes: bool(1), float64(2), int64(2), object(9)
memory usage: 205.7+ KB
